<a href="https://colab.research.google.com/github/coffepowered/fun-with-nlp/blob/text-vectorization/embedding_with_textvectorization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#